In [22]:
import pandas as pd
k=1000

data = pd.read_csv("GWAS.csv")
features = data.iloc[:, 1:]
variances = features.var()
top_5000_variances = variances.nlargest(k)
selected_features = features[top_5000_variances.index]
selected_data = pd.concat([data.iloc[:, 0], selected_features], axis=1)
selected_data.to_csv('{}.csv'.format(k), index=False)
data_df = pd.read_csv('{}.csv'.format(k))
for i in range(5):
    train_df = pd.read_csv("cross{}/train.csv".format(i))
    test_df = pd.read_csv("cross{}/test.csv".format(i))
    valid_df = pd.read_csv("cross{}/valid.csv".format(i))
    for df, filename in zip([train_df, test_df, valid_df], ["train", "test", "valid"]):
        filenames = df["filename"].tolist()
        for j in range(len(filenames)):
            filenames[j] = filenames[j][:-4]
        subset_df = data_df[data_df["index"].isin(filenames)]
        subset_df.to_csv("cross{}/GWAS_{}.csv".format(i,filename), index=False)

In [23]:
for i in range(5):
    for j in ['train','test','valid']:
        train_df = pd.read_csv("cross{}/{}.csv".format(i,j))
        subset_df = pd.read_csv("cross{}/GWAS_{}.csv".format(i,j))
        train_df = train_df.iloc[:,[1,5]]
        dict_map = {}
        for m in range(len(train_df)):
            train_df.iloc[m,0]=train_df.iloc[m,0][:-4]
            dict_map[train_df.iloc[m,0]]=train_df.iloc[m,1]
        s=[]
        for n in range(len(train_df)):
            s.append(dict_map[subset_df.iloc[n,0]])
        subset_df['COG']=s
        subset_df.to_csv('cross{}/GWAS_{}.csv'.format(i,j),index=False)

In [34]:
import json
def read_json(config_file):
    with open(config_file) as config_buffer:
        config = json.loads(config_buffer.read())
    return config
file = read_json('/root/autodl-tmp/ncomms2022-main/config/GWAS.json')

subset_df = pd.read_csv("cross{}/GWAS_{}.csv".format(0,'train'))

In [35]:
file['features'][0] in subset_df.columns

True

In [4]:
import pandas as pd
for i in range(5):
    for j in ['train','valid','test']:
        data = pd.read_csv('../../tb_log/CNN_baseline_new_cross{}/{}_eval.csv'.format(i,j))
        new = pd.read_csv('cross{}/{}.csv'.format(i,j))
        
        new['COG_score']=data['COG_score']
        if 'ADD_score' in data.columns:
            new['ADD_score']=data['ADD_score']
        new.to_csv('cross{}/{}_mri.csv'.format(i,j),index=False)